In [1]:
# !git clone https://github.com/edgarschnfld/CADA-VAE-PyTorch
!git clone https://github.com/Sieg-Arash/MEVAE

Cloning into 'MEVAE'...
remote: Enumerating objects: 106, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 106 (delta 24), reused 55 (delta 16), pack-reused 42
Receiving objects: 100% (106/106), 214.68 MiB | 37.84 MiB/s, done.
Resolving deltas: 100% (33/33), done.


In [2]:
# !pip install -r /kaggle/working/graduation-project-/CADAVAE/requirements.txt

In [3]:
!rm -rf DCA-VAE/.git

In [4]:
cd /kaggle/working/MEVAE

/kaggle/working/MEVAE


In [5]:
!mkdir data

In [6]:
cd data

/kaggle/working/MEVAE/data


In [7]:
!wget https://www.dropbox.com/sh/btoc495ytfbnbat/AAAaurkoKnnk0uV-swgF-gdSa?dl=0

--2021-05-26 00:49:50--  https://www.dropbox.com/sh/btoc495ytfbnbat/AAAaurkoKnnk0uV-swgF-gdSa?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.18, 2620:100:601f:18::a27d:912
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/raw/btoc495ytfbnbat/AAAaurkoKnnk0uV-swgF-gdSa [following]
--2021-05-26 00:49:50--  https://www.dropbox.com/sh/raw/btoc495ytfbnbat/AAAaurkoKnnk0uV-swgF-gdSa
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uce5dce1c14abaea3a98af15854c.dl.dropboxusercontent.com/zip_download_get/AynkWWFzdHuuVmmzJ6GytDoqmZsEZXivslkgEuX33Fsslpk4Ex3vg93ZtIHeLWo_3ELZ_xhn5Sl9tvT-ALFz7ftN9o2p8_QTJagHzzJJUu7M_Q# [following]
--2021-05-26 00:49:51--  https://uce5dce1c14abaea3a98af15854c.dl.dropboxusercontent.com/zip_download_get/AynkWWFzdHuuVmmzJ6GytDoqmZsEZXivslkgEuX33Fsslpk4Ex3vg93ZtIHeLWo_3ELZ_xhn5Sl9tvT-

In [8]:
!mv AAAaurkoKnnk0uV-swgF-gdSa?dl=0 data.zip

In [9]:
!unzip data.zip

Archive:  data.zip
mapname:  conversion of  failed
   creating: CUB/
   creating: SUN/
   creating: AWA2/
   creating: AWA1/
 extracting: SUN/res101.mat          
 extracting: CUB/res101.mat          
 extracting: AWA1/res101.mat         
 extracting: AWA2/res101.mat         
 extracting: SUN/att_splits.mat      
 extracting: CUB/att_splits.mat      
 extracting: AWA2/att_splits.mat     
 extracting: AWA1/att_splits.mat     
 extracting: CUB/CUB_supporting_data.p  


In [10]:
cd ..

/kaggle/working/MEVAE


In [11]:
!python3 single_experiment.py --dataset CUB --num_shots 0 --generalized False

***
22
The current working directory is
/kaggle/working/MEVAE
Project Directory:
/kaggle/working/MEVAE
Data Path
/kaggle/working/MEVAE/data
_____
/kaggle/working/MEVAE/data/CUB/res101.mat
resnet_features 2048
attributes 312
/opt/conda/lib/python3.7/site-packages/torch/optim/adam.py:48: UserWarning: optimizer contains a parameter group with duplicate parameters; in future, this will cause an error; see github.com/pytorch/pytorch/issues/40967 for more information
  super(Adam, self).__init__(params, defaults)
/opt/conda/lib/python3.7/site-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
train for reconstruction
epoch 0 | iter 0	 | loss 8494.
epoch 0 | iter 50	 | loss 5280.
epoch 0 | iter 100	 | loss 5009.
epoch 1 | iter 0	 | loss 4668.
epoch 1 | iter 50	 | loss 4255.
epoch 1 | iter 100	 | loss 4017.
epoch 2 | iter 0	 | loss 4345.
epoch 2 | iter 50	 | loss 3904.
epoch 

In [12]:
from vaemodel import Model
import numpy as np
import pickle
import torch
import os
import argparse


saved_state = torch.load('./CADA_trained.pth.tar')
hyperparameters = saved_state['hyperparameters']
model = Model(hyperparameters)
model.load_state_dict(saved_state['state_dict'])
for d in model.all_data_sources:
    model.encoder[d].load_state_dict(saved_state['encoder'][d])
    model.decoder[d].load_state_dict(saved_state['decoder'][d])
    
u,s,h,history = model.train_classifier()


if hyperparameters['generalized']==True:
    acc = [hi[2] for hi in history]
elif hyperparameters['generalized']==False:
    acc = [hi[1] for hi in history]

print(acc[-1])


FileNotFoundError: [Errno 2] No such file or directory: './CADA_trained.pth.tar'

In [ ]:
import sys
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader,TensorDataset
import numpy as np
import torch
import torch.tensor as tensor
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import time
from sklearn.manifold import TSNE
%matplotlib inline
def plot_embedding(data, label, title):
    x_min, x_max = np.min(data, 0), np.max(data, 0)
    data = (data - x_min) / (x_max - x_min)

    fig = plt.figure()
    ax = plt.subplot(111)
    for i in range(data.shape[0]):
        plt.text(data[i, 0], data[i, 1], str(label[i]),
                 color=plt.cm.Set1(label[i]/50),
                 fontdict={'weight': 'bold', 'size': 9})
    plt.xticks([])
    plt.yticks([])
    plt.title(title)
    plt.savefig("tsne.svg", format='svg')
    plt.show()
def plot_confusion_matrix(cm, savename, title='Confusion Matrix', classes = None):

    plt.figure(figsize=(12, 8), dpi=100)
    np.set_printoptions(precision=2)

    # 在混淆矩阵中每格的概率值
    ind_array = np.arange(len(classes))
    x, y = np.meshgrid(ind_array, ind_array)
#     for x_val, y_val in zip(x.flatten(), y.flatten()):
#         c = cm[y_val][x_val]
#         if c > 0.001:
#             plt.text(x_val, y_val, "%0.2f" % (c,), color='red', fontsize=15, va='center', ha='center')
    
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.binary)
    plt.title(title)
    plt.colorbar()
    xlocations = np.array(range(len(classes)))
    plt.xticks(xlocations, classes, rotation=90)
    plt.yticks(xlocations, classes)
    plt.ylabel('Actual label')
    plt.xlabel('Predict label')
    
    # offset the tick
    tick_marks = np.array(range(len(classes))) + 0.5
    plt.gca().set_xticks(tick_marks, minor=True)
    plt.gca().set_yticks(tick_marks, minor=True)
    plt.gca().xaxis.set_ticks_position('none')
    plt.gca().yaxis.set_ticks_position('none')
    plt.grid(True, which='minor', linestyle='-')
    plt.gcf().subplots_adjust(bottom=0.15)
    
    # show confusion matrix
    plt.savefig(savename, format='svg')
    plt.show()
t0 = time.time()
test_seen_X=np.load("test_seen_X.npy")
test_novel_X=np.load("test_novel_X.npy")
print(test_seen_X.shape, test_novel_X.shape)
tsne = TSNE(n_components=2, init='pca', random_state=0)
# test_seen_X_tsne = tsne.fit_transform(test_seen_X)
test_novel_X_tsne = tsne.fit_transform(test_novel_X)
np.save("test_novel_X_tsne.npy",test_novel_X_tsne)
# test_novel_X_tsne = np.load("test_novel_X_tsne.npy")
predicted_label_zsl=np.load("predicted_label_zsl.npy")
original_label_zsl=np.load("original_label_zsl.npy")
original_label_zsl = np.array(original_label_zsl, dtype='int')
print(predicted_label_zsl.shape)
print(original_label_zsl.shape)
plot_embedding(test_novel_X_tsne, original_label_zsl,
                     't-SNE embedding of the digits (time %.2fs)'
                     % (time.time() - t0))
unique_labels = np.unique(original_label_zsl)
cm = confusion_matrix(original_label_zsl, predicted_label_zsl)
print(cm.max())
max_cm = cm.max()
localtime = time.asctime( time.localtime(time.time()) )
plot_confusion_matrix(cm/max_cm, localtime[-13:-5]+".svg", title='confusion matrix', classes = unique_labels)